## 2. The impact of Scaling the Dataset

The only change here is to modify the FMNISTDataset class where the input value is divided by the maximum possible value in the dataset which is 255. This to ensure that all the variables values are between 0 and 1.

### Import The Relevant Packages

In [13]:
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

###  Download and Prepare The Dataset

In [14]:
from torchvision import datasets
data_folder = "Datasets"                        
fmnist = datasets.FashionMNIST(data_folder, download=True, train=True)

In [15]:
train_images = fmnist.data                                
train_targets = fmnist.targets                            

In [16]:
class FMNISTDataset(Dataset):
    def __init__(self,x,y):
        x= x.float()/255            #divide the input data by the maximum value
        x = x.view(-1,28*28)        
        self.x , self.y = x,y
    def __len__(self):
        return len(self.x)
    def __getitem__(self,index):
        x,y = self.x[index], self.y[index]
        return x.to(device), y.to(device)  

In [ ]:
def build_model(optimizer = SGD, lr = 1e-2 ):
    
    model = nn.Sequential(
                nn.Linear(28 * 28, 1000),
                nn.ReLU(),
                nn.Linear(1000,10)
                         ).to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr = lr)   
    
    return model , loss_function, optimizer


In [ ]:
def train_batch(input_batch, targets, model, loss_function, optimizer):
    model.train()                                        
    predictions = model(input_batch)                     #perfom forward propagation
    batch_loss = loss_function(predictions,targets)      #compute the loss value 
    batch_loss.backward()                                #perfom backward propagation
    optimizer.step()                                     #calculate new weights
    optimizer.zero_grad()                                #flush the gradients for next batch of calculations
    
    return batch_loss.item()                             #return the loss value

In [17]:
train_data = FMNISTDataset(train_images,train_targets) 
train_data_loader = DataLoader(train_data, batch_size=32, shuffle=True)